In [ ]:
# !pip install pdf2image

In [ ]:
# !pip install fugashi

In [ ]:
# !pip install unidic-lite

In [ ]:
# !pip install langchain

In [ ]:
# !pip install paddlepaddle


In [43]:
from pdf2image import convert_from_path
from paddleocr import PaddleOCR
import os
from langchain_community.llms import Ollama

In [44]:
pdf_ls = os.listdir("pdf")

In [ ]:
# import os
# from pdf2image import convert_from_path

pdf_ls = os.listdir("pdf")
# Path to the PDF file
for pdf in pdf_ls:
    pdf_path = f"pdf/{pdf}"
    if not os.path.exists("Docs"):
        os.mkdir("Docs")
    doc_path = f"Docs/{pdf_path[4:-4:]}"
    if not os.path.exists(doc_path):
        os.mkdir(doc_path)
    # Convert PDF pages to images
    images = convert_from_path(pdf_path)

    # Save each image
    for i, img in enumerate(images):
        img.save(f"{doc_path}/page_{i+1:02d}.jpg")

    # Print a success message
    print(f"{len(images)} images were successfully extracted from {pdf_path}.")


In [48]:
llm = Ollama(model="mixtral")

In [51]:
cleaned_ls = []
ocr = PaddleOCR(use_angle_cls=True, lang='en') # need to run only once to load model into memory
doc_ls = sorted(os.listdir("Docs/"))
for i, doc in enumerate(doc_ls):
    page_ls = sorted(os.listdir(f"Docs/{doc}"))
    print(f"processing document ----> Docs/{doc}")
    strings = []
    for page in page_ls:
        print(f"processing page ----> Docs/{doc}/{page}")
        img_path = os.path.join(f"Docs/{doc}", page)
        # print(img_path)
        result = ocr.ocr(img_path, cls=True)
        for idx in range(len(result)):
            res = result[idx]
            for line in res:
                print("<----Extracted Text---->\n",line[1][0])
                strings.append(line[1][0])
    final_doc = ('\n').join(strings)
    print(final_doc)
    cleaned_txt = llm.invoke(f"{final_doc}; convert the given document in the format of Minutes of meaning")
    print("<----Cleaned Text---->\n", cleaned_txt)
    cleaned_ls.append(cleaned_txt)
    if i > 6:
        break
print(len(cleaned_ls))

[2024/04/26 13:25:06] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/Users/divyeshpatel/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/divyeshpatel/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_len

In [ ]:
# final_doc = ('\n').join(strings)

In [ ]:
# print(final_doc)

# summarize the strings and do embeggings and make retrieval chains

In [52]:
master_context = ('\n\n').join(cleaned_ls)
# Open the file in write mode
with open('output.txt', 'w') as file:
    # Write a string to the file
    file.write(master_context)

In [53]:
analysis_doc = llm.invoke(f"read the given document {master_context}; and write an analysis report on that")

In [54]:
# Open the file in write mode
with open('analysis_report.txt', 'w') as file:
    # Write a string to the file
    file.write(analysis_doc)

In [ ]:
# clean_ls = []

In [ ]:
# llm = Ollama(model="mistral")

# cleaned_txt = llm.invoke(f"clean this {final_doc}")

# clean_ls.append(cleaned_txt)

# print(cleaned_txt)